# Flow

## 1. Persiapan Data:

### 1.a Buat matriks jarak antara semua pasangan port.
### 1.b Tentukan jumlah kendaraan dan kapasitas masing-masing kendaraan.
### 1.c Buat matriks pheromone awal.


In [18]:

# Konfigurasi koneksi ke database
dbname = "route_optimization"
user = "postgres"
password = "password"
host = "localhost"
port = 54321


db_params = {
    'dbname': 'route_optimization',
    'user': 'postgres',
    'password': 'password',
    'host': 'localhost',
    'port': '54321',
    'options': '-c search_path=route_optimization'
}


# Get Data from database

In [28]:
import psycopg2
import psycopg2.extras as extras
# Membuka koneksi
#connection = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
connection = psycopg2.connect(**db_params)
# Membuat objek cursor
cursor = connection.cursor()

# Contoh query SELECT
query = "SELECT origin_code, destination_code, nautical_distance FROM master_nautical_distance_2"
cursor.execute(query)
results = cursor.fetchall()

# Menampilkan hasil query
data_list = []
for row in results:
    orig_value, dest_value , distance= row
    data_list.append((orig_value, dest_value,distance))

# Menampilkan daftar hasil query
#print(data_list)

# Menutup cursor dan koneksi
cursor.close()
connection.close()

# membuat matrix jarak

In [27]:
#data = [('pc1', 'pc1', 0.0), ('pc2', 'pc1', 796.0), ('pc3', 'pc1', 1380.0)]

# Find unique port codes
port_codes = set()
for item in data_list:
    port_codes.add(item[0])
    port_codes.add(item[1])

port_codes = sorted(port_codes)
num_ports = len(port_codes)

# Initialize distance matrix
distance_matrix = [[0.0] * num_ports for _ in range(num_ports)]

# Fill in distance matrix using data
for item in data_list:
    origin_index = port_codes.index(item[0])
    destination_index = port_codes.index(item[1])
    distance = item[2]
    distance_matrix[origin_index][destination_index] = distance

# Display distance matrix
for row in distance_matrix:
    print(row)


[0.0, 625.0, 292.0, 1162.0, 507.0, 560.0, 1390.0, 2932.0, 920.0, 1337.0, 325.0, 796.0, 1893.0, 1527.0, 1366.0, 2626.0, 856.0, 1532.0, 2088.0, 1551.0, 1683.0, 1753.0, 1380.0, 766.0, 2786.0, 1462.0, 1730.0, 1468.0, 927.0, 1121.0, 1633.0, 380.0, 1587.0, 2599.0, 2636.0, 1654.0, 811.0, 835.0, 2683.0, 2152.0, 2531.0, 1972.0, 1412.0, 1574.0, 1576.0, 1178.0, 1619.0, 1769.0, 921.0, 608.0, 2484.0, 1147.0, 2863.0, 707.0, 1363.0, 1137.0, 2524.0, 2155.0, 1449.0, 2556.0, 612.0, 110.0, 1813.0, 752.0, 401.0, 1303.0, 1027.0, 1784.0, 1223.0, 1047.0, 1494.0, 2419.0, 730.0, 1792.0, 1958.0, 1065.0, 1659.0, 2988.0, 1744.0, 1496.0, 1340.0, 1284.0, 2303.0, 1875.0, 0.0]
[625.0, 0.0, 301.0, 717.0, 153.0, 65.0, 657.0, 2390.0, 556.0, 1003.0, 299.0, 135.0, 587.0, 779.0, 447.0, 2084.0, 535.0, 588.0, 1546.0, 776.0, 599.0, 1081.0, 318.0, 488.0, 2244.0, 714.0, 611.0, 540.0, 604.0, 700.0, 618.0, 1005.0, 839.0, 2114.0, 2094.0, 1654.0, 329.0, 293.0, 2141.0, 1128.0, 2247.0, 946.0, 905.0, 1032.0, 1034.0, 636.0, 1111.0, 728

In [3]:
import pandas as pd
import numpy as np

# Membuka koneksi
connection = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)

# Membuat objek cursor
cursor = connection.cursor()

# Contoh query SELECT
query = "SELECT origin_code, destination_code, nautical_distance FROM route_optimization.master_nautical_distance"
#cursor.execute(query)
#results = cursor.fetchall()


# Baca hasil query ke dalam DataFrame pandas
df = pd.read_sql(query, connection)

# Menampilkan DataFrame
print("DataFrame:")
print(df)

# Buat daftar unik kode port dari origin_code dan destination_code
unique_ports = np.unique(df[['origin_code', 'destination_code']])

# Buat matriks jarak
num_ports = len(unique_ports)
distance_matrix = np.zeros((num_ports, num_ports))


C:\Users\User\AppData\Local\Temp\ipykernel_13828\4163406465.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


DataFrame:
    origin_code destination_code  nautical_distance
0        port_1          port_50              380.0
1        port_2          port_10               95.0
2        port_3          port_23              289.0
3        port_3          port_10              132.0
4        port_3          port_71              384.0
..          ...              ...                ...
181     port_96          port_39              104.0
182     port_97           port_3              283.0
183     port_97          port_33              124.0
184     port_98          port_54              127.0
185     port_99          port_60               30.0

[186 rows x 3 columns]
Matrix Jarak:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [4]:
# jarak nautical dari csv
import pandas as pd

# Read the CSV file into a DataFrame
df_nautical = pd.read_csv('jarak-nautical.csv')

# Display the DataFrame
# print(df)

destination_port = pd.read_csv('jarak-nautical.csv', nrows=0).columns.tolist()
origin_port = pd.read_csv('jarak-nautical.csv', nrows=0).columns.tolist()
destination_port.remove('PORT')
origin_port.remove('PORT')

# Print the header
print(origin_port)

['pc1', 'pc2', 'pc3', 'pc4', 'pc5', 'pc6', 'pc7', 'pc8', 'pc9', 'pc10', 'pc11', 'pc12', 'pc13', 'pc14', 'pc15', 'pc16', 'pc17', 'pc18', 'pc19', 'pc20', 'pc21', 'pc22', 'pc23', 'pc24', 'pc25', 'pc26', 'pc27', 'pc28', 'pc29', 'pc30', 'pc31', 'pc32', 'pc33', 'pc34', 'pc35', 'pc36', 'pc37', 'pc38', 'pc39', 'pc40', 'pc41', 'pc42', 'pc43', 'pc44', 'pc45', 'pc46', 'pc47', 'pc48', 'pc49', 'pc50', 'pc51', 'pc52', 'pc53', 'pc54', 'pc55', 'pc56', 'pc57', 'pc58', 'pc59', 'pc60', 'pc61', 'pc62', 'pc63', 'pc64', 'pc65', 'pc66', 'pc67', 'pc68', 'pc69', 'pc70', 'pc71', 'pc72', 'pd1', 'pd2', 'pd3', 'pd4', 'pd5', 'pd6', 'pd7', 'pd8', 'pd9', 'pd10', 'pd11', 'pd12']


In [29]:
# Create an index on the 'Name' column
df_nautical2 = df_nautical.set_index('PORT')


## Sudah di insert 
string_insert = " "

insert_query = "INSERT INTO master_nautical_distance_2 (origin_code, destination_code, nautical_distance, comercial_distance) VALUES"


for item_destination in destination_port:
    for item_origin in origin_port:
        distance =  df_nautical2.at[item_destination, item_origin]
        #string_insert += "(origin_code = '{}', destination_code='{}', nautical_distance ={}, comercial_distance={}),".format (item_origin,item_destination,distance,distance )
        string_insert += "('{}', '{}', {}, {}),".format (item_origin,item_destination,distance,distance )
        

string_insert +="( 'zzzzz', 'zzzzz', 0, 0)  "      

insert_query +=string_insert
print(insert_query)

try:
    # Connect to the database
    #conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
    conn = psycopg2.connect(**db_params)

    cursor = conn.cursor()

    # Execute the INSERT statement
    cursor.execute(insert_query)

    # Commit the transaction
    conn.commit()

    print("Insert successful!")

except psycopg2.Error as e:
    print("Error:", e)

finally:
    # Close the cursor and connection
    if cursor:
        cursor.close()
    if conn:
        conn.close()



In [ ]:
#list port
# Contoh data port dalam bentuk daftar tuple (kode_port, nama_port)
# Daftar port dengan kode_port dan nama_port
port_data = [
    {'kode_port': 'A', 'nama_port': 'Port A'},
    {'kode_port': 'B', 'nama_port': 'Port B'},
    {'kode_port': 'C', 'nama_port': 'Port C'},
    # Tambahkan entri lain sesuai kebutuhan
]

# Kode port yang ingin dicari namanya
kode_port_cari = 'B'

# Mencari nama_port berdasarkan kode_port
nama_port = next((port['nama_port'] for port in port_data if port['kode_port'] == kode_port_cari), None)

if nama_port is not None:
    print(f"Nama port dengan kode {kode_port_cari} adalah {nama_port}.")
else:
    print(f"Tidak ditemukan nama port untuk kode {kode_port_cari}.")


In [ ]:
# Inisialisasi pheromone pada setiap jalur
pheromone_matrix = np.ones((len(origin_ports), len(destination_ports)))

# Algoritma Koloni Semut
for iteration in range(max_iterations):
    # Simulasi pergerakan semut
    for vehicle in range(num_vehicles):
        # Inisialisasi semut di kota awal (misalnya, kota 0)
        current_city = vehicle * (num_cities // num_vehicles)
        unvisited_cities = set(range(num_cities))
        unvisited_cities.remove(current_city)

        # Inisialisasi rute kendaraan
        route = [current_city]

        # Pergi ke kota berikutnya hingga semua kota dikunjungi
        while unvisited_cities:
            next_city = min(
                unvisited_cities,
                key=lambda city: pheromone_matrix[current_city][city]
                / distance_matrix[current_city][city],
            )
            route.append(next_city)
            unvisited_cities.remove(next_city)
            current_city = next_city

        # Kembali ke kota awal untuk menutup rute kendaraan
        route.append(vehicle * (num_cities // num_vehicles))

        # Hitung total panjang rute kendaraan
        total_distance = sum(distance_matrix[route[i]][route[i + 1]] for i in range(len(route) - 1))

        # Update pheromone pada rute yang baru saja dilalui oleh kendaraan
        for i in range(len(route) - 1):
            pheromone_matrix[route[i]][route[i + 1]] += 1.0 / total_distance

    # Penguapan pheromone pada setiap jalur setelah semua kendaraan telah bergerak
    pheromone_matrix *= 0.9

# Menampilkan hasil rute untuk masing-masing kendaraan
for vehicle in range(num_vehicles):
    start_city = vehicle * (num_cities // num_vehicles)
    end_city = (vehicle + 1) * (num_cities // num_vehicles)
    print(f"Rute Kendaraan {vehicle+1}: {route[start_city:end_city+1]}")